# Passange Survival by Fare paid on HMS Titanic 

[Kaggle Competition](https://www.kaggle.com/c/titanic)

In [0]:
# EDA
import numpy as np
import modin.pandas as pd
import missingno
import seaborn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pandas.plotting import scatter_matrix

# preprocessing
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import sklearn.decomposition

# algos
import xgboost as xgb
import 

In [0]:
# load data
!wget https://raw.githubusercontent.com/erinkhoo/House_pricing_dataset/master/dataset_csv/train.csv
!wget https://raw.githubusercontent.com/erinkhoo/House_pricing_dataset/master/dataset_csv/test.csv